# Dataset Preparation

In this notebook, we will prepare our datasets in a form that is suitable to the application of machine learning algorithms.

In [2]:
from pymatgen.core.composition import Composition
import numpy as np
import pandas as pd
import ase.db # https://wiki.fysik.dtu.dk/ase/ase/db/db.html
from itertools import product

In [3]:
def extract_elements(elements, atomic_table):
    # Extracts the rows of `atomic_table` corresponding to `elements`
    return atomic_table[atomic_table['Element'].isin(elements)]

def construct_features_row(dict,atomic_table):
    """
    Constructs a feature row that will be used in the machine learning algorithms;
    The features are statistical functions involving the properties of all atoms of a molecule;
    `dict` is a dictionary that represents the chemical formula of the material;
    """
    
    #Extracts the atomic properties of relevant elements
    df = extract_elements(dict.keys(),atomic_table)

    #Converts them to a numpy array
    array = df.drop(columns=df.columns[:1], axis=1).to_numpy()

    #Mean
    mean = np.mean(array,axis=0)

    #Weighted Mean
    copy = np.copy(array)

    natoms = sum(dict.values())

    for (n,w) in enumerate(dict.values()):
        copy[n,:] *= w / natoms
            
    wmean = np.sum(copy,axis=0) 

    #Maximum
    max = np.max(array,axis=0)

    #Minimum
    min = np.min(array,axis=0)

    #Standard deviation with respect to mean
    std = np.std(array,axis=0)

    #Standard deviation with respect to weighted mean
    copy = np.copy(array)
    
    for (n,m) in enumerate(wmean):
        copy[:,n] -= m

    wstd = np.mean(copy**2, axis = 0)

    return np.hstack((mean,wmean,max,min,std,wstd))


def construct_features(data, atomic_table):
    # Constructs the entire dataset one row at a time, simply iterating through the previous function

    #Builds the name of all columns
    columns=['Material', 'Space Group', 'gap']
    for term in ('_mean', '_wmean', '_max', '_min', '_std', '_wstd'):
        columns.extend((atomic_table.columns[1:] + term))
    
    df = pd.DataFrame(columns=columns)

    for row in data:
        new_entry = [row.formula, row.spacegroup, row['gap']]
        dict = Composition(row.formula).as_dict()
        new_entry.extend(construct_features_row(dict,atomic_table))            
        df.loc[len(df)] = new_entry

    return df

In [4]:
atomic_table = pd.read_csv('Schleder2019_AtomicTable.csv')
atomic_table.fillna(0,inplace=True)
atomic_table.head()

,Element,Z,Electronegativity,IonizationPotential,ElectronAffinity,HOMO,LUMO,r_s_orbital,r_p_orbital,r_d_orbital,r_atomic_nonbonded,r_valence_lastorbital,r_covalent,Valence,PeriodicColumn,PeriodicColumn_upto18,NumberUnfilledOrbitals,Polarizability
0,H,1,2.20,-12.6833,-1.5273,-6.4925,0.7250,0.3865,0.0000,0.0000,0.37,0.3865,0.31,1.0,1.0,1.0,1.0,4.507107
1,He,2,0.00,-26.7499,3.0204,-15.7610,1.5714,0.2964,1.0292,0.4176,0.32,0.2964,0.28,2.0,8.0,18.0,0.0,1.383746
2,Li,3,0.98,-5.3606,-0.5863,-2.8744,-0.9074,1.6578,1.8874,2.0869,1.34,1.6578,1.28,1.0,1.0,1.0,1.0,164.000000
3,Be,4,1.57,-9.5007,0.7972,-5.6097,-2.0104,1.0805,1.2128,1.9594,0.90,1.0805,0.96,2.0,2.0,2.0,0.0,37.710000
4,B,5,2.04,-8.1261,0.0312,-3.6067,2.4547,0.8025,0.8348,1.3619,0.82,0.8348,0.84,3.0,3.0,13.0,5.0,20.530000


In [5]:
data = ase.db.connect('c2db-2021-06-24.db').select(is_magnetic=False)

In [6]:
df = construct_features(data, atomic_table)

In [6]:
df.head()

,Material,Space Group,gap,Z_mean,Electronegativity_mean,IonizationPotential_mean,ElectronAffinity_mean,HOMO_mean,LUMO_mean,r_s_orbital_mean,...,r_p_orbital_wstd,r_d_orbital_wstd,r_atomic_nonbonded_wstd,r_valence_lastorbital_wstd,r_covalent_wstd,Valence_wstd,PeriodicColumn_wstd,PeriodicColumn_upto18_wstd,NumberUnfilledOrbitals_wstd,Polarizability_wstd
0,Be4,Pbcm,0.000000,4.0,1.570,-9.50070,0.79720,-5.60970,-2.01040,1.08050,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,AlTe4,Cm,0.000000,32.5,1.855,-7.70680,-1.20540,-4.33185,3.55250,1.02700,...,0.020793,0.557753,0.009826,0.020793,0.009826,57.460000,3.060000,3.060000,3.060000,115.110400
2,As4O6,P2_1,3.876239,20.5,2.810,-12.70170,-2.20980,-7.11345,4.20300,0.65830,...,0.099108,0.175934,0.055016,0.099108,0.073034,21.060000,0.260000,0.260000,0.260000,156.830336
3,As4S6,Pc,2.271995,24.5,2.380,-10.41110,-2.10070,-6.10240,4.41855,0.80155,...,0.010546,0.338489,0.007514,0.010546,0.005096,21.060000,0.260000,0.260000,0.260000,28.284074
4,B2N,P-3m1,0.000000,6.0,2.540,-10.84585,-0.82285,-5.34825,3.30365,0.66930,...,0.028338,0.006076,0.001361,0.028338,0.004694,1.111111,1.111111,1.111111,1.111111,46.440250


In [7]:
df.to_csv('gap_prediction.csv')

# Preparation of a novel dataset

In [7]:
#List with all possible space groups
space_groups=set(pd.read_csv('gap_prediction.csv')["Space Group"])

metals=['Sc','Ti','Cr','Mn','Fe','Co','Ni','Cu','Zn']
halogens=['F','Cl','Br','I']
stoichiometries=[[2,3],[1,2],[1,3]]

def construct_novel_dataset(atomic_table):
    columns=['Material', 'Space Group']
    for term in ('_mean', '_wmean', '_max', '_min', '_std', '_wstd'):
        columns.extend((atomic_table.columns[1:] + term))

    df = pd.DataFrame(columns=columns)

    for (space_group,metal,halogen,stoichiometry) in product(space_groups,metals,halogens,stoichiometries):
        formula = dict(zip([metal,halogen],stoichiometry))
        new_entry = [Composition(formula).reduced_formula, space_group]
        new_entry.extend(construct_features_row(formula,atomic_table))            
        df.loc[len(df)] = new_entry

    return df

In [8]:
data = ase.db.connect('c2db-2021-06-24.db').select(is_magnetic=False)
novel_dataset = construct_novel_dataset(atomic_table)

In [9]:
novel_dataset

,Material,Space Group,Z_mean,Electronegativity_mean,IonizationPotential_mean,ElectronAffinity_mean,HOMO_mean,LUMO_mean,r_s_orbital_mean,r_p_orbital_mean,...,r_p_orbital_wstd,r_d_orbital_wstd,r_atomic_nonbonded_wstd,r_valence_lastorbital_wstd,r_covalent_wstd,Valence_wstd,PeriodicColumn_wstd,PeriodicColumn_upto18_wstd,NumberUnfilledOrbitals_wstd,Polarizability_wstd
0,Sc2F3,Pmmn,15.0,2.670,-13.15365,-1.62310,-7.17485,0.76665,1.02750,0.99850,...,0.405600,0.061058,0.138554,0.011130,0.331994,4.160000,4.160000,50.960000,16.640000,2774.431400
1,ScF2,Pmmn,15.0,2.670,-13.15365,-1.62310,-7.17485,0.76665,1.02750,0.99850,...,0.433334,0.065233,0.148028,0.011891,0.354694,4.444444,4.444444,54.444444,17.777778,2964.136111
2,ScF3,Pmmn,15.0,2.670,-13.15365,-1.62310,-7.17485,0.76665,1.02750,0.99850,...,0.487500,0.073387,0.166531,0.013377,0.399031,5.000000,5.000000,61.250000,20.000000,3334.653125
3,Sc2Cl3,Pmmn,19.0,2.260,-10.44055,-1.42390,-5.91520,2.36990,1.16455,1.18985,...,0.195124,0.153354,0.052650,0.008035,0.120224,4.160000,4.160000,50.960000,16.640000,2221.259274
4,ScCl2,Pmmn,19.0,2.260,-10.44055,-1.42390,-5.91520,2.36990,1.16455,1.18985,...,0.208465,0.163840,0.056250,0.008585,0.128444,4.444444,4.444444,54.444444,17.777778,2373.140250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5935,ZnBr2,P-4,32.5,2.305,-11.19935,-0.75910,-6.91785,2.31675,0.92910,1.26345,...,0.160487,0.001424,0.008028,0.013863,0.000111,6.944444,6.944444,6.944444,0.277778,79.017210
5936,ZnBr3,P-4,32.5,2.305,-11.19935,-0.75910,-6.91785,2.31675,0.92910,1.26345,...,0.180548,0.001602,0.009031,0.015596,0.000125,7.812500,7.812500,7.812500,0.312500,88.894361
5937,Zn2I3,P-4,41.5,2.155,-10.49295,-0.66035,-6.53055,1.72050,1.00460,1.35920,...,0.084060,0.003273,0.000104,0.000265,0.007514,6.500000,6.500000,6.500000,0.260000,4.298413
5938,ZnI2,P-4,41.5,2.155,-10.49295,-0.66035,-6.53055,1.72050,1.00460,1.35920,...,0.089807,0.003497,0.000111,0.000283,0.008028,6.944444,6.944444,6.944444,0.277778,4.592321


In [10]:
novel_dataset.to_csv('gap_prediction_novel.csv')